# MUBen: Usage Demonstration

In this demonstration, we'll guide you through a foundational training and testing of MUBen using the BBBP dataset as a basic example.
We've chosen the DNN as our backbone model because it is both efficient and offers satisfactory performance.
For uncertainty quantification (UQ), we'll evaluate both the Deterministic method (referred to as "none" within MUBen) and Temperature Scaling.
While the procedures for other backbone models, UQ methods, or datasets are largely similar, you can explore specific variations by referring to the scripts in the `<project root>/run/` directory.

In [1]:
# Import the necessary packages
import sys
sys.path.append('../')
import logging
import wandb
from transformers import set_seed

from muben.dnn.dataset import Dataset
from muben.dnn.args import Config
from muben.dnn.train import Trainer
from muben.utils.io import set_logging

# initialize logger
logger = logging.getLogger(__name__)

## Run the Deterministic UQ method

In [2]:
# Set up the logging format and random seed.
# We do not use wandb for this demo, so we set its mode to "disabled".
set_logging()
set_seed(0)
wandb.init(mode="disabled",)

# Specify the configuration of the experiment.
# Notice that although we directly edit the config object here, a more appropriate way of doing this is 
# passing arguments through the shell or json scripts when we are running the experiments through the terminal.
config = Config()
config.model_name = "DNN"
config.feature_type = "rdkit"
config.data_folder = "../data/files/"
config.dataset_name = "bbbp"
config.result_folder = "../output-demo/"
config.uncertainty_method = "none"
config.retrain_model = True

config.n_epochs = 20  # We only train the model for a small number of epochs for the demo.
config.time_training = True

config.__post_init__()

# Load dataset metadata, validate the arguments, and log the configuration.
_ = config.get_meta().validate().log()


08/23/2023 17:34:59 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
08/23/2023 17:34:59 - INFO - muben.base.args -   Configurations: (<class 'muben.dnn.args.Config'>)
08/23/2023 17:34:59 - INFO - muben.base.args -     apply_preconditioned_sgld: False
08/23/2023 17:34:59 - INFO - muben.base.args -     apply_temperature_scaling_after_focal_loss: False
08/23/2023 17:34:59 - INFO - muben.base.args -     apply_wandb: True
08/23/2023 17:34:59 - INFO - muben.base.args -     batch_size: 32
08/23/2023 17:34:59 - INFO - muben.base.args -     batch_size_inference: None
08/23/2023 17:34:59 - INFO - muben.base.args -     bbp_prior_sigma: 0.1
08/23/2023 17:34:59 - INFO - muben.base.args -     binary_classification_with_softmax: False
08/23/2023 17:34:59 - INFO - muben.base.args -     classes: [0, 1]
08/23/2023 17:34:59 - INFO - muben.base.args -     d_dnn_hidden: 128
08/23/2023 17

In [3]:
# Load and process the training, validation and test datasets
training_dataset = Dataset().prepare(config=config, partition="train")
valid_dataset = Dataset().prepare(config=config, partition="valid")
test_dataset = Dataset().prepare(config=config, partition="test")

08/23/2023 17:34:59 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/train.pt
08/23/2023 17:34:59 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/valid.pt
08/23/2023 17:34:59 - INFO - muben.base.dataset -   Loading pre-processed dataset ../data/files/bbbp/processed/DNN-rdkit/test.pt


In [4]:
# Inintialized the trainer with the configuration and datasets
trainer = Trainer(
    config=config,
    training_dataset=training_dataset,
    valid_dataset=valid_dataset,
    test_dataset=test_dataset,
)

08/23/2023 17:34:59 - INFO - muben.base.train.trainer -   Trainer initialized. The model contains 157953 parameters


In [5]:
# Run the training, validation and test process.
# The model checkpoint and predicted results will be automatically saved in the specified output folder.
trainer.run()

08/23/2023 17:34:59 - INFO - muben.base.train.trainer -   Training model
08/23/2023 17:34:59 - INFO - muben.base.train.trainer -   [Training Epoch 0]
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -     Training Loss: 0.5374.
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -     Average Time per Step: 0.0241.
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -   [Valid step 1] results:
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -     roc_auc: 0.3002.
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -   Model buffer is updated!
08/23/2023 17:35:01 - INFO - muben.base.train.trainer -   [Training Epoch 1]
08/23/2023 17:35:02 - INFO - muben.base.train.trainer -     Training Loss: 0.4387.
08/23/2023 17:35:02 - INFO - muben.base.train.trainer -     Average Time per Step: 0.0180.
08/23/2023 17:35:02 - INFO - muben.base.train.trainer -   [Valid step 2] results:
08/23/2023 17:35:02 - INFO - muben.base.train.trainer -     roc_auc: 0.8225.
08/23/2023 17:35:02 - IN

## Run Temperature Scaling

In [6]:
wandb.init(mode="disabled",)
# Change some configuration items.
config.uncertainty_method = "TemperatureScaling"
config.retrain_model = False
config.n_ts_epochs = 10  # number of epochs for training the temperature scaling layer.
config.__post_init__()
_ = config.validate().log()

08/23/2023 17:35:20 - INFO - muben.base.args -   Configurations: (<class 'muben.dnn.args.Config'>)
08/23/2023 17:35:20 - INFO - muben.base.args -     apply_preconditioned_sgld: False
08/23/2023 17:35:20 - INFO - muben.base.args -     apply_temperature_scaling_after_focal_loss: False
08/23/2023 17:35:20 - INFO - muben.base.args -     apply_wandb: True
08/23/2023 17:35:20 - INFO - muben.base.args -     batch_size: 32
08/23/2023 17:35:20 - INFO - muben.base.args -     batch_size_inference: None
08/23/2023 17:35:20 - INFO - muben.base.args -     bbp_prior_sigma: 0.1
08/23/2023 17:35:20 - INFO - muben.base.args -     binary_classification_with_softmax: False
08/23/2023 17:35:20 - INFO - muben.base.args -     classes: [0, 1]
08/23/2023 17:35:20 - INFO - muben.base.args -     d_dnn_hidden: 128
08/23/2023 17:35:20 - INFO - muben.base.args -     d_feature: 200
08/23/2023 17:35:20 - INFO - muben.base.args -     data_dir: ../data/files/bbbp
08/23/2023 17:35:20 - INFO - muben.base.args -     data_

In [7]:
# Re-inintialized the trainer with the updated configuration.
# The datasets are not changed.
trainer = Trainer(
    config=config,
    training_dataset=training_dataset,
    valid_dataset=valid_dataset,
    test_dataset=test_dataset,
)

08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   Trainer initialized. The model contains 157953 parameters


In [8]:
# Run the training, validation and test process.
# The trainer will load the model checkpoint from the Deterministic run and
# continue training the temperature scaling layer.
# Notice that not all UQ methods support continued training. For example, BBP requires training from scratch.
trainer.run()

08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   Loading trained model from ../output-demo/bbbp/DNN-rdkit/TemperatureScaling/seed-0/model_best.ckpt.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   Temperature Scaling session start.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   Training model on validation
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   [Training Epoch 0]
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -     Training Loss: 0.5309.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -     Average Time per Step: 0.0090.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   [Training Epoch 1]
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -     Training Loss: 0.5303.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -     Average Time per Step: 0.0059.
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -   [Training Epoch 2]
08/23/2023 17:35:20 - INFO - muben.base.train.trainer -     Training Loss: 0.5300.


## Calculating Metrics
This is a simplified version of metric calculation. Please check `<project root>/assist/result_get_metrics.py` for the full function.

In [9]:
import os.path as op
import pandas as pd
from muben.utils.metrics import classification_metrics
from muben.utils.io import load_results

In [20]:
# Define the path to the predicted results. "det" stands for "Deterministic"; "ts" stands for "Temperature Scaling".
det_result = op.join(
    config.result_folder, config.dataset_name, f"{config.model_name}-{config.feature_type}",
    "none", f"seed-{config.seed}", "preds", "0.pt"
)
ts_result = op.join(
    config.result_folder, config.dataset_name, f"{config.model_name}-{config.feature_type}",
    "TemperatureScaling", f"seed-{config.seed}", "preds", "0.pt"
)

# Load the predicted results.
det_preds, _, lbs, masks = load_results([det_result])
ts_preds, _, _, _ = load_results([ts_result])

In [21]:
# Calculate the metrics.
det_metrics = classification_metrics(det_preds, lbs, masks)
ts_metrics = classification_metrics(ts_preds, lbs, masks)

det_metrics = {k: v['macro-avg'] for k, v in det_metrics.items()}
ts_metrics = {k: v['macro-avg'] for k, v in ts_metrics.items()}

In [24]:
# Present the results in a dataframe.
det_metrics_df = pd.DataFrame({"Deterministic": det_metrics, "TemperatureScaling": ts_metrics})
det_metrics_df.T

                     roc-auc   prc-auc       ece       mce       nll     brier
Deterministic       0.685378  0.739983  0.233773  0.606446  0.864909  0.285275
TemperatureScaling  0.685378  0.739983  0.224520  0.529412  0.827177  0.279270
